# README.md

###For each pt with an ADT duration, find all T test results within 

# Step1, read from spreadsheet, clean up, and get statistics

In [104]:
import numpy as np
import pandas as pd
import sys, getopt
import csv
from datetime import datetime
from datetime import timedelta, date
import itertools

from matplotlib import pyplot
from pylab import *
import matplotlib.pyplot as plt

%matplotlib inline  
import seaborn as sns

In [105]:
#df1=pd.read_excel("bigcohort_bothok_new.xlsx", header=0)
df1.fillna('', inplace=True)
#df1['inital_ADT_end_date'] = pd.to_datetime(df1['inital_ADT_end_date'])

df1=pd.read_csv("bigcohort_bothok_new.csv", header=0)
df1.fillna('', inplace=True)

df2=pd.read_excel("pca_tresults_20210119_BP_fixed.xlsx", header=0)
#df3=pd.read_csv("just_start_empty.csv", header=0)

In [106]:
df1[:5]

,person_id,inital_ADT_start_date,inital_ADT_end_date,Inital_ADT_timewindow,Inital_ADT_time_window_days,index1
0,2,2009-02-26,2017-02-01,2897 days 00:00:00.000000000,2897.0,0
1,13,2010-06-17,2018-05-17,2891 days 00:00:00.000000000,2891.0,2
2,18,2013-11-12,2014-03-13,121 days 00:00:00.000000000,121.0,3
3,22,2010-09-27,2017-04-24,2401 days 00:00:00.000000000,2401.0,4
4,25,2013-11-12,2014-02-18,98 days 00:00:00.000000000,98.0,5


In [107]:
df2[:5]

,person_id,cancer_type_id,year_of_diagnosis,month_of_diagnosis,day_of_diagnosis,dx_date,result_date,loinc_code,source_test_name,test_results
0,2,2,2003,1.0,7.0,2003-01-07,2006-09-05,49041-7,TESTOSTERONE,351.0
1,2,2,2003,1.0,7.0,2003-01-07,2007-01-09,49041-7,TESTOSTERONE,304.0
2,2,2,2003,1.0,7.0,2003-01-07,2007-05-17,49041-7,TESTOSTERONE,259.0
3,2,2,2003,1.0,7.0,2003-01-07,2007-06-12,49041-7,TESTOSTERONE,382.0
4,2,2,2003,1.0,7.0,2003-01-07,2007-06-28,49041-7,TESTOSTERONE,439.0


In [108]:
adt_pt_list=df1["person_id"].to_list()

df_tvalue_grouped=df2.groupby(["person_id"]).size().reset_index(name='counts')
list_pt_test=df_tvalue_grouped["person_id"].to_list()
print("")

overlap_list=[x for x in adt_pt_list if x in  list_pt_test]

print("pt number in the adt table is:")
print(len(adt_pt_list))
print("")
print("pt number in the T value table is:")
print(len(list_pt_test))
print("")
print("pt number overlapping is:")
print(len(overlap_list))


pt number in the adt table is:
940

pt number in the T value table is:
4490

pt number overlapping is:
940


In [151]:
index_list=df1.columns
index_list

Index(['person_id', 'inital_ADT_start_date', 'inital_ADT_end_date',
       'Inital_ADT_timewindow', 'Inital_ADT_time_window_days', 'index1'],
      dtype='object')

In [152]:
index2_list=df2.columns
index2_list

Index(['person_id', 'cancer_type_id', 'year_of_diagnosis',
       'month_of_diagnosis', 'day_of_diagnosis', 'dx_date', 'result_date',
       'loinc_code', 'source_test_name', 'test_results'],
      dtype='object')

In [153]:
y=2

df1_pt=df1[(df1["person_id"] == y)]
df2_pt=df2[(df2["person_id"] == y)]

In [154]:
df1_pt

,person_id,inital_ADT_start_date,inital_ADT_end_date,Inital_ADT_timewindow,Inital_ADT_time_window_days,index1
0,2,2009-02-26,2017-02-01,2897 days 00:00:00.000000000,2897.0,0


In [155]:
df1_pt.inital_ADT_start_date

0   2009-02-26
Name: inital_ADT_start_date, dtype: datetime64[ns]

In [156]:
df2_pt

,person_id,cancer_type_id,year_of_diagnosis,month_of_diagnosis,day_of_diagnosis,dx_date,result_date,loinc_code,source_test_name,test_results
0,2,2,2003,1.0,7.0,2003-01-07,2006-09-05,49041-7,TESTOSTERONE,351.00
1,2,2,2003,1.0,7.0,2003-01-07,2007-01-09,49041-7,TESTOSTERONE,304.00
2,2,2,2003,1.0,7.0,2003-01-07,2007-05-17,49041-7,TESTOSTERONE,259.00
3,2,2,2003,1.0,7.0,2003-01-07,2007-06-12,49041-7,TESTOSTERONE,382.00
4,2,2,2003,1.0,7.0,2003-01-07,2007-06-28,49041-7,TESTOSTERONE,439.00
...,...,...,...,...,...,...,...,...,...,...
77,2,2,2003,1.0,7.0,2003-01-07,2017-02-01,49041-7,TESTOSTERONE,15.64
78,2,2,2003,1.0,7.0,2003-01-07,2017-02-22,49041-7,TESTOSTERONE,7.72
79,2,2,2003,1.0,7.0,2003-01-07,2017-03-18,49041-7,TESTOSTERONE,11.29
80,2,2,2003,1.0,7.0,2003-01-07,2017-03-22,49041-7,TESTOSTERONE,11.14


In [157]:
def closest(lst, K): 
      return lst[min(range(len(lst)), key = lambda i: abs(lst[i] - K))] 

In [158]:
df2_pt=df2[(df2["person_id"] == 2)]    
each_pt_tdates = df2_pt['result_date'].to_list()
#each_pt_tdates

In [159]:
adt_pt_list

[2,
 13,
 18,
 22,
 25,
 26,
 37,
 38,
 81,
 95,
 119,
 125,
 132,
 137,
 153,
 267,
 275,
 300,
 323,
 341,
 386,
 387,
 776,
 780,
 800,
 870,
 905,
 939,
 955,
 1181,
 1229,
 1231,
 1238,
 1257,
 1280,
 1503,
 1538,
 1583,
 1611,
 1618,
 1657,
 1658,
 1704,
 1881,
 1883,
 1887,
 1910,
 1934,
 1936,
 1964,
 2058,
 2061,
 2063,
 2088,
 2200,
 2237,
 2256,
 2278,
 2507,
 2526,
 2557,
 2561,
 2590,
 2605,
 2623,
 2628,
 2652,
 2655,
 2660,
 2683,
 2713,
 3030,
 3079,
 3084,
 3085,
 3129,
 3131,
 3155,
 3177,
 3181,
 3354,
 3385,
 3396,
 3415,
 3419,
 3574,
 3576,
 3578,
 3618,
 3619,
 3666,
 3667,
 3697,
 3854,
 3874,
 3905,
 3920,
 3951,
 3976,
 3993,
 3995,
 4056,
 4075,
 4208,
 4211,
 4231,
 4235,
 4255,
 4257,
 4275,
 4316,
 4318,
 4319,
 4562,
 4595,
 4626,
 4627,
 4667,
 4670,
 4717,
 4763,
 4793,
 4795,
 4854,
 4855,
 4876,
 4959,
 5014,
 5021,
 5041,
 5042,
 5043,
 5049,
 5052,
 5181,
 5229,
 5496,
 5498,
 5561,
 5630,
 5761,
 5763,
 5771,
 5791,
 5870,
 5872,
 5873,
 5875,
 590

In [160]:
df1['inital_ADT_end_date'] = pd.to_datetime(df1['inital_ADT_end_date'])
df1['inital_ADT_start_date'] = pd.to_datetime(df1['inital_ADT_start_date'])

In [161]:
#pd.to_datetime(df1_pt['inital_ADT_end_date'], utc=False)
a=df1_pt['inital_ADT_start_date'].to_list()[0]
#df.dates.values.astype('M8[D]')
a

Timestamp('2009-02-26 00:00:00')

In [162]:
df1_pt=df1[(df1["person_id"] == 25)]
df1_pt

,person_id,inital_ADT_start_date,inital_ADT_end_date,Inital_ADT_timewindow,Inital_ADT_time_window_days,index1
4,25,2013-11-12,2014-02-18,98 days 00:00:00.000000000,98.0,5


In [163]:
if df1_pt.inital_ADT_end_date.empty:
    print("fine")
if len(df1_pt.inital_ADT_end_date)==0:
    print("fine")

# Match T test date and value just before ADT <a class="anchor" id="second-bullet"></a>

In [165]:
result_before_list=[]

#for y in [12427]:
for y in adt_pt_list: 
    
    df1_pt=df1[(df1["person_id"] == y)]
    
    #if(df1_pt.inital_ADT_end_date.values.any()==""):  
    if len(df1_pt.inital_ADT_end_date)==0:
        a='NaN'
        b='NaN'
        gap='NaN'
        tdate='NaN'
        tvalue='NaN'
        N_t_dates='NaN'
        N_t_dates_afterADT='NaN'
    
    #if(df1_pt.inital_ADT_start_date.values.any()!=""):
    if len(df1_pt.inital_ADT_start_date)!=0:
        a=df1_pt['inital_ADT_start_date'].to_list()[0]
        adt_end=df1_pt['inital_ADT_end_date'].to_list()[0] 
        #pick up initial ADT window 
        gap=df1_pt['Inital_ADT_time_window_days'].to_list()[0] 
    
        df2_pt=df2[(df2["person_id"] == y)]    
        each_pt_tdates = df2_pt['result_date'].to_list()

        ##get rid of nan
        cleaned_a=[]
        cleaned_a= [x for x in each_pt_tdates if str(x) != 'NaN']
        cleaned_aa= [x for x in cleaned_a if str(x) != 'nan']
        cleaned_aaa= [x for x in cleaned_aa if str(x) != 'NaT']
        N_t_dates=len(cleaned_aaa)

        #only keep the ones before ADT start date, including on the day
        cleaned_aaaa= [x for x in cleaned_aaa if x <a]
        
        start_sorted_bin=sorted(cleaned_aaaa)
        N_t_dates_beforeADT=len(cleaned_aaaa)
    
        if(len(start_sorted_bin)==0):
            tdate='NaN' 
            tvalue="NaN"

        if(len(start_sorted_bin)>0):
            #apply the closet function
            #tdate=closest(start_sorted_bin,a_plus_6mons)
            tdate=closest(start_sorted_bin,a)
            #tvalue=df2_pt.loc[df['result_date'] == tdate].test_result
    
            #pick up T-value for T-date
            df2_pt_pick=df2_pt[(df2_pt['result_date']==tdate)]
            tvalue=df2_pt_pick.test_results.values[0]
    
    print(y, a, gap,tdate, tvalue,N_t_dates,N_t_dates_beforeADT)
    result_per_pt=[y, a, adt_end, gap, tdate, tvalue,N_t_dates,N_t_dates_beforeADT]
    result_before_list.append(result_per_pt)

2 2009-02-26 00:00:00 2897.0 2009-02-09 00:00:00 274.18 82 20
13 2010-06-17 00:00:00 2891.0 2009-09-18 00:00:00 23.72 23 2
18 2013-11-12 00:00:00 121.0 NaN NaN 5 0
22 2010-09-27 00:00:00 2401.0 NaN NaN 3 0
25 2013-11-12 00:00:00 98.0 2013-10-15 00:00:00 31.54 12 1
26 2014-11-26 00:00:00 553.0 NaN NaN 14 0
37 2010-09-29 00:00:00 2165.0 NaN NaN 23 0
38 2015-11-18 00:00:00 243.0 2015-10-28 00:00:00 222.13 20 4
81 2013-09-25 00:00:00 188.0 NaN NaN 12 0
95 2011-01-05 00:00:00 1204.0 NaN NaN 25 0
119 2013-07-02 00:00:00 92.0 2013-06-17 00:00:00 403.1 12 2
125 2016-01-25 00:00:00 1682.0 NaN NaN 18 0
132 2015-04-13 00:00:00 457.0 NaN NaN 1 0
137 2014-02-01 00:00:00 485.0 2013-07-05 00:00:00 276.45 19 1
153 2014-07-02 00:00:00 2051.0 NaN NaN 2 0
267 2015-05-19 00:00:00 359.0 NaN NaN 35 0
275 2019-12-09 00:00:00 281.0 2019-05-23 00:00:00 252.0 12 9
300 2016-09-17 00:00:00 379.0 NaN NaN 14 0
323 2015-09-09 00:00:00 364.0 2015-07-22 00:00:00 26.74 7 1
341 2016-05-18 00:00:00 366.0 2016-04-26 00:00

6656 2016-06-20 00:00:00 192.0 NaN NaN 14 0
6794 2019-04-04 00:00:00 58.0 2019-02-25 00:00:00 342.96 3 1
6841 2011-10-03 00:00:00 3248.0 NaN NaN 6 0
6846 2015-03-03 00:00:00 1407.0 NaN NaN 14 0
6894 2004-04-02 00:00:00 3848.0 NaN NaN 10 0
6933 2016-02-01 00:00:00 1260.0 NaN NaN 1 0
6955 2015-06-15 00:00:00 366.0 NaN NaN 22 0
6958 2015-02-26 00:00:00 861.0 NaN NaN 8 0
6963 2019-06-07 00:00:00 455.0 2015-04-16 00:00:00 171.54 1 1
6997 2019-01-18 00:00:00 669.0 2014-10-10 00:00:00 146.9 1 1
7207 2016-11-04 00:00:00 649.0 NaN NaN 5 0
7236 2016-08-19 00:00:00 286.0 2016-08-03 00:00:00 698.89 24 1
7237 2018-07-17 00:00:00 815.0 NaN NaN 1 0
7268 2018-06-25 00:00:00 672.0 2018-06-18 00:00:00 269.03 8 1
7295 2014-03-31 00:00:00 701.0 2011-12-07 00:00:00 504.2 2 2
7532 2017-08-01 00:00:00 1123.0 2017-04-21 00:00:00 355.87 5 3
7565 2019-08-19 00:00:00 256.0 NaN NaN 3 0
7611 2015-02-17 00:00:00 1303.0 2015-01-28 00:00:00 212.29 34 5
7615 2011-07-18 00:00:00 1004.0 2011-06-09 00:00:00 466.3 12 4
76

17638 2012-07-03 00:00:00 790.0 2012-04-11 00:00:00 370.65 4 1
17662 2013-12-19 00:00:00 2477.0 2013-12-05 00:00:00 293.88 40 1
17687 2012-11-01 00:00:00 1145.0 NaN NaN 1 0
17709 2013-09-16 00:00:00 28.0 2013-09-05 00:00:00 26.97 25 3
17710 2014-04-01 00:00:00 1191.0 2014-03-25 00:00:00 351.14 49 2
17759 2014-09-05 00:00:00 266.0 2014-08-08 00:00:00 11.71 13 1
17782 2015-07-23 00:00:00 1362.0 2015-07-08 00:00:00 308.29 30 2
17793 2014-08-05 00:00:00 290.0 NaN NaN 16 0
17865 2014-08-01 00:00:00 138.0 NaN NaN 2 0
18010 2018-08-13 00:00:00 80.0 NaN NaN 1 0
18025 2018-01-22 00:00:00 994.0 NaN NaN 10 0
18031 2017-02-01 00:00:00 425.0 NaN NaN 3 0
18071 2017-09-29 00:00:00 1022.0 NaN NaN 9 0
18225 2015-01-01 00:00:00 1540.0 NaN NaN 3 0
18253 2016-07-18 00:00:00 1164.0 2016-05-24 00:00:00 350.22 33 5
18303 2017-02-06 00:00:00 941.0 2017-01-27 00:00:00 249.58 9 1
18356 2015-02-25 00:00:00 2056.0 2014-03-05 00:00:00 25.27 3 2
18357 2014-05-02 00:00:00 307.0 NaN NaN 31 0
18362 2018-05-31 00:00:00

28209 2010-12-23 00:00:00 207.0 2010-06-28 00:00:00 333.57 32 18
28219 2019-04-01 00:00:00 127.0 NaN NaN 1 0
28250 2018-08-21 00:00:00 773.0 NaN NaN 1 0
28445 2015-04-09 00:00:00 747.0 NaN NaN 14 0
28452 2019-07-30 00:00:00 91.0 NaN NaN 2 0
28474 2014-05-05 00:00:00 637.0 NaN NaN 7 0
28496 2010-05-01 00:00:00 1047.0 NaN NaN 1 0
28630 2017-07-28 00:00:00 931.0 2017-07-18 00:00:00 266.25 23 2
28634 2016-11-17 00:00:00 1394.0 NaN NaN 7 0
28662 2014-03-04 00:00:00 91.0 2014-02-06 00:00:00 23.76 7 2
28709 2014-08-28 00:00:00 407.0 NaN NaN 14 0
28710 2012-07-12 00:00:00 3049.0 NaN NaN 11 0
28712 2018-04-24 00:00:00 91.0 2017-04-21 00:00:00 585.32 1 1
28714 1999-05-01 00:00:00 5442.0 NaN NaN 13 0
28716 2016-06-13 00:00:00 756.0 2016-05-23 00:00:00 141.72 32 17
28769 2014-02-14 00:00:00 630.0 NaN NaN 6 0
28827 2014-03-01 00:00:00 404.0 NaN NaN 7 0
28829 2014-09-23 00:00:00 1182.0 NaN NaN 15 0
29029 2014-09-05 00:00:00 853.0 2014-08-08 00:00:00 20.8 15 1
29117 2015-07-20 00:00:00 748.0 NaN NaN 

171441 2018-09-01 00:00:00 744.0 NaN NaN 18 0
171442 2018-12-03 00:00:00 241.0 NaN NaN 18 0
171741 2019-07-24 00:00:00 448.0 2019-07-19 00:00:00 455.33 9 1
171789 2019-06-03 00:00:00 476.0 NaN NaN 1 0
172448 2019-09-19 00:00:00 196.0 NaN NaN 5 0
173013 2019-11-12 00:00:00 93.0 NaN NaN 2 0
173883 2019-08-09 00:00:00 416.0 2019-07-12 00:00:00 256.47 9 2
174044 2019-01-28 00:00:00 648.0 NaN NaN 2 0
174314 2020-01-22 00:00:00 176.0 NaN NaN 2 0
174400 2019-09-13 00:00:00 362.0 2019-08-05 00:00:00 543.02 8 1
174453 2019-02-12 00:00:00 407.0 2019-02-11 00:00:00 401.69 6 1
174677 2019-07-24 00:00:00 406.0 2019-07-15 00:00:00 318.87 8 1
175751 2019-11-06 00:00:00 271.0 NaN NaN 2 0
175984 2019-12-11 00:00:00 278.0 2019-10-17 00:00:00 88.0 3 1
176201 2019-03-25 00:00:00 527.0 2019-03-11 00:00:00 188.91 9 1
176291 2019-09-30 00:00:00 283.0 2019-06-06 00:00:00 408.0 4 1
176393 2019-06-12 00:00:00 84.0 NaN NaN 3 0
176612 2018-03-19 00:00:00 74.0 NaN NaN 14 0
177083 2020-05-19 00:00:00 94.0 NaN NaN 1

In [166]:
df_results_before_adt=pd.DataFrame(result_before_list)
df_results_before_adt.columns=(['person_id', 'inital_ADT_start_date', 'inital_ADT_end_date', 'Inital_ADT_time_window_days', 'Tdate_justbeforeADT', 'Tvalue_justbeforeADT', "N_T_dates","N_T_dates_beforeADT"])

In [167]:
len(df_results_before_adt)

940

# Match T test date and value Immediately after ADT <a class="anchor" id="third-bullet"></a>

In [168]:
result_after_list=[]

for y in adt_pt_list: 
#for y in [12427]:
#for y in list_pts_adt_table: 
    
    df1_pt=df1[(df1["person_id"] == y)]
    
    if len(df1_pt.inital_ADT_end_date)==0:    
        b='NaN'
        gap='NaN'
        tdate='NaN'
        tvalue='NaN'
        N_t_dates='NaN'
        N_t_dates_afterADT='NaN'

    if len(df1_pt.inital_ADT_end_date)!=0:    
        b=df1_pt['inital_ADT_end_date'].to_list()[0]
        df2_pt=df2[(df2["person_id"] == y)]    
        each_pt_tdates = df2_pt['result_date'].to_list()

        ##get rid of nan
        cleaned_a=[]
        cleaned_a= [x for x in each_pt_tdates if str(x) != 'NaN']
        cleaned_aa= [x for x in cleaned_a if str(x) != 'nan']
        cleaned_aaa= [x for x in cleaned_aa if str(x) != 'NaT']
        N_t_dates=len(cleaned_aaa)

        #only keep the ones after ADT end date including on the day
        cleaned_aaaa= [x for x in cleaned_aaa if x >b]
        start_sorted_bin=sorted(cleaned_aaaa)
        N_t_dates_afterADT=len(cleaned_aaaa)
    
        if(len(start_sorted_bin)==0):
            tdate='NaN' 
            tvalue="NaN"

        if(len(start_sorted_bin)>0):
            #apply the closet function
            #tdate=closest(start_sorted_bin,a_plus_6mons)
            tdate=closest(start_sorted_bin,b)
            #tvalue=df2_pt.loc[df['result_date'] == tdate].test_result
    
            #pick up T-value for T-date
            df2_pt_pick=df2_pt[(df2_pt['result_date']==tdate)]
            tvalue=df2_pt_pick.test_results.values[0]
    
    print(y, b, tdate, tvalue ,N_t_dates,N_t_dates_afterADT)
    result_after_per_pt=[y,b,tdate, tvalue ,N_t_dates,N_t_dates_afterADT]
    result_after_list.append(result_after_per_pt)

2 2017-02-01 00:00:00 2017-02-22 00:00:00 7.72 82 4
13 2018-05-17 00:00:00 NaN NaN 23 0
18 2014-03-13 00:00:00 2014-12-02 00:00:00 31.3 5 5
22 2017-04-24 00:00:00 NaN NaN 3 0
25 2014-02-18 00:00:00 2015-02-18 00:00:00 149.89 12 11
26 2016-06-01 00:00:00 2016-06-15 00:00:00 15.75 14 7
37 2016-09-02 00:00:00 2016-09-27 00:00:00 14.74 23 1
38 2016-07-18 00:00:00 2016-09-26 00:00:00 11.51 20 12
81 2014-04-01 00:00:00 2014-04-10 00:00:00 313.07 12 12
95 2014-04-23 00:00:00 2014-04-28 00:00:00 10.0 25 1
119 2013-10-02 00:00:00 2013-11-15 00:00:00 18.24 12 10
125 2020-09-02 00:00:00 NaN NaN 18 0
132 2016-07-13 00:00:00 2016-10-20 00:00:00 8.41 1 1
137 2015-06-01 00:00:00 2016-04-12 00:00:00 459.52 19 16
153 2020-02-12 00:00:00 NaN NaN 2 0
267 2016-05-12 00:00:00 2016-05-23 00:00:00 7.77 35 28
275 2020-09-15 00:00:00 NaN NaN 12 0
300 2017-10-01 00:00:00 2018-02-21 00:00:00 6.9 14 10
323 2016-09-07 00:00:00 NaN NaN 7 0
341 2017-05-19 00:00:00 NaN NaN 39 0
386 2017-02-01 00:00:00 2017-02-23 00:0

11250 2020-08-24 00:00:00 NaN NaN 19 0
11276 2012-09-17 00:00:00 NaN NaN 2 0
11284 2020-10-23 00:00:00 NaN NaN 1 0
11461 2014-01-01 00:00:00 NaN NaN 2 0
11482 2020-04-23 00:00:00 2020-05-21 00:00:00 5.08 33 1
11543 2016-11-01 00:00:00 2016-11-23 00:00:00 35.9 8 1
11565 2020-11-16 00:00:00 NaN NaN 18 0
11595 2017-10-25 00:00:00 NaN NaN 6 0
11613 2019-01-02 00:00:00 2019-01-03 00:00:00 13.0 5 1
11628 2020-10-12 00:00:00 NaN NaN 6 0
11649 2017-12-11 00:00:00 2018-05-04 00:00:00 14.47 7 5
11651 2020-11-06 00:00:00 NaN NaN 4 0
11709 2018-08-01 00:00:00 2018-11-14 00:00:00 18.3 1 1
11797 2017-10-31 00:00:00 2018-04-30 00:00:00 1083.86 1 1
11821 2012-05-07 00:00:00 2012-09-27 00:00:00 10.0 11 2
11832 2019-12-10 00:00:00 NaN NaN 10 0
11861 2020-10-12 00:00:00 NaN NaN 3 0
11876 2020-09-25 00:00:00 NaN NaN 29 0
11907 2019-12-01 00:00:00 NaN NaN 2 0
11930 2014-07-30 00:00:00 NaN NaN 5 0
11933 2017-12-29 00:00:00 NaN NaN 3 0
12031 2016-08-04 00:00:00 2016-09-27 00:00:00 4.33 9 1
12116 2017-12-06 0

21523 2020-03-28 00:00:00 NaN NaN 4 0
21536 2019-07-16 00:00:00 2019-10-15 00:00:00 19.0 3 1
21543 2014-05-05 00:00:00 2014-11-03 00:00:00 39.73 11 11
21571 2019-03-07 00:00:00 NaN NaN 9 0
21606 2020-10-26 00:00:00 NaN NaN 5 0
21613 2017-10-12 00:00:00 2018-04-16 00:00:00 31.0 11 3
21618 2019-11-28 00:00:00 NaN NaN 2 0
21655 2016-05-31 00:00:00 2016-06-30 00:00:00 15.64 11 8
21656 2016-06-01 00:00:00 2016-06-22 00:00:00 11.15 21 16
21762 2019-07-24 00:00:00 2019-11-06 00:00:00 519.0 11 2
21764 2013-11-25 00:00:00 2014-08-12 00:00:00 27.58 8 6
21779 2020-08-28 00:00:00 NaN NaN 2 0
21831 2020-03-09 00:00:00 NaN NaN 16 0
21832 2018-01-01 00:00:00 NaN NaN 10 0
21932 2019-01-07 00:00:00 NaN NaN 16 0
21938 2019-12-19 00:00:00 NaN NaN 8 0
22100 2018-01-17 00:00:00 2018-04-18 00:00:00 43.98 16 7
22103 2019-07-16 00:00:00 2020-03-30 00:00:00 86.0 4 1
22329 2020-08-04 00:00:00 NaN NaN 2 0
22330 2018-09-17 00:00:00 2019-04-24 00:00:00 61.41 2 1
22334 2019-09-26 00:00:00 NaN NaN 3 0
22380 2020-08-

31921 2016-03-22 00:00:00 2016-06-24 00:00:00 7.16 19 10
31927 2016-10-05 00:00:00 NaN NaN 2 0
31933 2020-09-29 00:00:00 NaN NaN 1 0
32081 2016-03-15 00:00:00 NaN NaN 23 0
32113 2014-08-13 00:00:00 2014-09-15 00:00:00 12.1 10 1
32116 2019-09-24 00:00:00 2019-12-10 00:00:00 3.0 12 1
32191 2017-11-01 00:00:00 NaN NaN 1 0
32230 2019-06-22 00:00:00 2020-01-13 00:00:00 161.53 16 1
32231 2019-06-22 00:00:00 NaN NaN 2 0
32277 2013-12-30 00:00:00 NaN NaN 2 0
32280 2016-02-26 00:00:00 NaN NaN 4 0
32421 2017-06-21 00:00:00 2018-11-28 00:00:00 322.92 7 2
32426 2020-07-07 00:00:00 NaN NaN 3 0
32660 2020-06-19 00:00:00 NaN NaN 19 0
32679 2020-02-03 00:00:00 2020-06-11 00:00:00 5.0 1 1
32740 2016-03-16 00:00:00 2016-09-14 00:00:00 231.23 5 3
32774 2020-01-24 00:00:00 NaN NaN 18 0
32797 2020-10-07 00:00:00 NaN NaN 13 0
32815 2011-09-30 00:00:00 2012-03-08 00:00:00 29.93 1 1
32877 2018-05-14 00:00:00 2019-01-11 00:00:00 592.0 1 1
33086 2020-05-13 00:00:00 2020-05-18 00:00:00 4.33 12 2
33106 2019-09-20

In [174]:
df_results_after_adt=pd.DataFrame(result_after_list)
#result_per_pt=[y, a, gap, tdate, tvalue]
#result_per_pt=[y, a, gap, tdate, tvalue,N_t_dates,N_t_dates_afterADT]
df_results_after_adt.columns=(['person_id','inital_ADT_end_date', 'Tdate immediately after ADT', 'Tvalue',"N_T_dates","N_T_dates_afterADT"])

In [175]:
len(df_results_after_adt)

940

In [176]:
df_results_before_adt.to_csv("940pts_ttest_results_before_adt.csv", encoding='utf-8', index=False)
df_results_after_adt.to_csv("940pts_ttest_results_after_adt.csv", encoding='utf-8', index=False)